## Import Libraries and Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
from sklearn import datasets
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.utils.data import Subset
import keras
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Concatenate, Dense, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.regularizers import L2
from keras.metrics import Precision, Recall, AUC
from keras.utils import to_categorical
from keras.optimizers import Adam
import pickle
import warnings
warnings.filterwarnings('ignore')

## Import Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os

# ADC data
data_path = '/content/drive/MyDrive/Computer Vision Project/Data/20230508'
file_names = os.listdir(data_path)
pkl_files = [file for file in file_names if file.endswith('.pkl')]
pkl_files = [pkl_files[0], pkl_files[2], pkl_files[4]]

file_path_y = data_path + '/' + pkl_files[0]
file_path_x = data_path + '/' + pkl_files[1]
file_path_idx = data_path + '/' + pkl_files[2]

with open(file_path_x, 'rb') as file:
  x = pickle.load(file)

with open(file_path_y, 'rb') as file:
  y = pickle.load(file)

with open(file_path_idx, 'rb') as file:
  idx = pickle.load(file)

Mounted at /content/drive


In [ ]:
pkl_files

['20230508_adc_annotated_label.pkl',
 '20230508_adc_input_rgb_128_all.pkl',
 '20230508_patients_index_updated.pkl']

In [ ]:
# Get a list of the keys
keys = list(idx.keys())

# Get the key at the desired index
train_key = keys[np.int(0.7*len(keys))]
print('Last index in training = ', idx[train_key][1])

val_key = keys[np.int(0.85*len(keys))]
print('Last index in validation = ', idx[val_key][1])

Last index in training =  19160
Last index in validation =  23234


## Split Training / Validation / Test

In [ ]:
# Get a list of the keys
keys = list(idx.keys())

# Get the keys at the desired index
train_key = keys[np.int(0.7*len(keys))] # outputs a list [start_idx, end_idx]
val_key = keys[np.int(0.85*len(keys))]
print(train_key, val_key)

# Split into training and validation and test
x_train = x[0:idx[train_key][1]]
y_train = y[0:idx[train_key][1]]

x_val = x[idx[train_key][1]:idx[val_key][1]]
y_val = y[idx[train_key][1]:idx[val_key][1]]

x_test = x[idx[val_key][1]:]
y_test = y[idx[val_key][1]:]

# Print shapes
print('x_train shape = ', x_train.shape)
print('y_train shape = ', y_train.shape)

print('x_val shape = ', x_val.shape)
print('y_val shape = ', y_val.shape)

print('x_test shape = ', x_test.shape)
print('y_test shape = ', y_test.shape)

11035_1001055 11256_1001279
x_train shape =  (19160, 128, 128, 3)
y_train shape =  (19160,)
x_val shape =  (4074, 128, 128, 3)
y_val shape =  (4074,)
x_test shape =  (4108, 128, 128, 3)
y_test shape =  (4108,)


In [ ]:
print(val_key)
print(idx[val_key])

11256_1001279
[23215, 23234]


## Redefine Class Weights

In [ ]:
import numpy as np
from sklearn.utils import class_weight
from collections import Counter

# Assuming you have your labels as a NumPy array or list
labels = y_train

# Find the sample sizes of each class
total_samples = x_train.shape[0]

def sample_per_class_counter(data):
  samples_per_class = []
  total_samples = data.shape[0]

  for class_label in np.unique(data):
    counter = 0
    for i in data:
      if i == class_label:
        counter += 1
    
    samples_per_class.append(counter)
  
  return samples_per_class

# Calculate class weights
sample_weights = class_weight.compute_sample_weight(class_weight='balanced', y=labels)
class_weights = np.unique(sample_weights)

# Create a dictionary mapping class index to class weight
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

print(class_weights_dict)

samples_per_class = sample_per_class_counter(y_train)
weights_per_class = torch.tensor([total_samples / class_sample_size for class_sample_size in samples_per_class], dtype=torch.float32)
weights_per_class

rescaling_factor = total_samples / torch.sum(weights_per_class)
weights_per_class *= rescaling_factor
weights_per_class_dict = {0: weights_per_class[0].item(), 1: weights_per_class[1].item()}
weights_per_class_dict

{0: 0.5177258971033291, 1: 14.603658536585366}


{0: 655.9999389648438, 1: 18504.0}

## Sanity Check for Distribution of Data

In [ ]:
# Find the sample sizes of each class (test)
samples_per_class_test = sample_per_class_counter(y_test)

samples_per_class_test = [100*x/y_test.shape[0] for x in samples_per_class_test]
samples_per_class_test

# Find the sample sizes of each class (train)
samples_per_class_train = sample_per_class_counter(y_train)

samples_per_class_train = [100*x/y_train.shape[0] for x in samples_per_class_train]

# Find the sample sizes of each class (validation)
samples_per_class_val = sample_per_class_counter(y_val)

samples_per_class_val = [100*x/y_val.shape[0] for x in samples_per_class_val]

print('Train distribution = ', samples_per_class_train)
print('Validation distribution = ', samples_per_class_val)
print('Test distribution = ', samples_per_class_test)

Train distribution =  [96.57620041753654, 3.4237995824634657]
Validation distribution =  [94.96809032891507, 5.031909671084929]
Test distribution =  [96.22687439143135, 3.7731256085686464]


## VGG19 : Raw data / No re-weighting / Pre-trained (No fine-tuning)

In [ ]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.1),
        keras.layers.RandomZoom(0.2),
        keras.layers.RandomContrast(0.8, 1.2),
        keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1)
    ]
)

In [ ]:
model = keras.applications.VGG19(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
)

model.trainable = True

#Adding custom Layers 
# Create the input tensor
inputs = keras.Input(shape=x_train.shape[1:])

# Apply data augmentation
augmented = data_augmentation(inputs)

# Apply normalization using the `Normalization` layer
normalized = keras.layers.Normalization()(augmented)

# Pass the normalized input through the VGG19 base model
x = model(normalized)

x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
output = keras.layers.Dense(1, activation="sigmoid", name="output")(x)

model_final = keras.Model(inputs =inputs, outputs = output)

model_final.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy',  metrics=[tf.keras.metrics.AUC()])

model_final.summary()

80134624/80134624 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 128, 128, 3)       0         
                                                                 
 normalization (Normalizatio  (None, 128, 128, 3)      7         
 n)                                                              
                                                                 
 vgg19 (Functional)          (None, None, None, 512)   20024384  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                              

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', # Metric to monitor
    patience=20,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True
    )  # Restore the weights of the best epoch

history = model_final.fit(x_train, y_train, 
                    batch_size=32, 
                    epochs=100,  # Increase the number of epochs to allow for early stopping
                    validation_split=0.2,
                    callbacks=early_stopping,
                    class_weight=class_weights_dict) 

Epoch 1/100
479/479 [==============================] - 28s 32ms/step - loss: 0.7624 - auc: 0.5915 - val_loss: 1.6581 - val_auc: 0.6171
Epoch 2/100
479/479 [==============================] - 14s 29ms/step - loss: 0.6373 - auc: 0.6757 - val_loss: 0.6721 - val_auc: 0.6743
Epoch 3/100
479/479 [==============================] - 14s 29ms/step - loss: 0.6588 - auc: 0.6365 - val_loss: 0.7360 - val_auc: 0.6171
Epoch 4/100
479/479 [==============================] - 14s 29ms/step - loss: 0.6409 - auc: 0.6732 - val_loss: 0.7168 - val_auc: 0.6911
Epoch 5/100
479/479 [==============================] - 14s 29ms/step - loss: 0.6371 - auc: 0.6716 - val_loss: 0.7519 - val_auc: 0.6632
Epoch 6/100
479/479 [==============================] - 14s 29ms/step - loss: 0.6304 - auc: 0.6854 - val_loss: 1.1414 - val_auc: 0.6788
Epoch 7/100
479/479 [==============================] - 14s 29ms/step - loss: 0.6220 - auc: 0.6977 - val_loss: 1.3193 - val_auc: 0.6712
Epoch 8/100
479/479 [==============================] - 

## Output Metrics for Training / Validation / Test

### Raw | Training

In [ ]:
y_pred_probs = model_final.predict(x_train)

def evaluate_thresholds(y_true, y_pred_probs, thresholds):
  results = []
  for threshold in thresholds:

      # Convert probabilities to binary predictions based on threshold
      y_pred = np.where(y_pred_probs >= threshold, 1, 0)
      
      # Calculate evaluation metrics
      positive_count = sum(y_pred)
      accuracy = accuracy_score(y_true, y_pred)
      precision = precision_score(y_true, y_pred)
      recall = recall_score(y_true, y_pred)
      f1 = f1_score(y_true, y_pred)
      auc = roc_auc_score(y_true, y_pred)
      
      results.append((threshold, positive_count, f1, precision, recall, auc, accuracy))
  
  results_df = pd.DataFrame(results, columns=["Threshold","positive_count" ,"F1", "Precision", "Recall", "AUC", "Accuracy"])
  return results_df

results_df = evaluate_thresholds(y_train, y_pred_probs, np.arange(0, 1, 0.05))
results_df

599/599 [==============================] - 6s 9ms/step


,Threshold,positive_count,F1,Precision,Recall,AUC,Accuracy
0,0.00,[19160],0.066209,0.034238,1.000000,0.500000,0.034238
1,0.05,[3819],0.177430,0.103954,0.605183,0.710125,0.807881
2,0.10,[3244],0.182051,0.109433,0.541159,0.692515,0.833507
3,0.15,[2873],0.187022,0.114863,0.503049,0.682810,0.850261
4,0.20,[2574],0.187616,0.117716,0.461890,0.669580,0.863048
5,0.25,[2341],0.191525,0.122597,0.437500,0.663248,0.873539
6,0.30,[2169],0.191150,0.124481,0.411585,0.654479,0.880741
7,0.35,[1993],0.194790,0.129453,0.393293,0.649765,0.888674
8,0.40,[1844],0.196800,0.133406,0.375000,0.644320,0.895198
9,0.45,[1685],0.199915,0.138872,0.356707,0.639146,0.902244


### Raw | Validation

In [ ]:
y_pred_probs = model_final.predict(x_val)

results_df = evaluate_thresholds(y_val, y_pred_probs, np.arange(0, 1, 0.05))
results_df

128/128 [==============================] - 1s 11ms/step


,Threshold,positive_count,F1,Precision,Recall,AUC,Accuracy
0,0.00,[4074],0.095817,0.050319,1.000000,0.500000,0.050319
1,0.05,[747],0.224790,0.143240,0.521951,0.678267,0.818851
2,0.10,[617],0.211679,0.141005,0.424390,0.643702,0.840943
3,0.15,[552],0.200793,0.137681,0.370732,0.623851,0.851497
4,0.20,[496],0.211127,0.149194,0.360976,0.625952,0.864261
5,0.25,[447],0.196319,0.143177,0.312195,0.606602,0.871379
6,0.30,[423],0.200637,0.148936,0.307317,0.607135,0.876780
7,0.35,[393],0.197324,0.150127,0.287805,0.600739,0.882180
8,0.40,[363],0.190141,0.148760,0.263415,0.591775,0.887089
9,0.45,[332],0.193669,0.156627,0.253659,0.590644,0.893716


### Raw | Test

In [ ]:
y_pred_probs = model_final.predict(x_test)

results_df = evaluate_thresholds(y_test, y_pred_probs, np.arange(0, 1, 0.05))
results_df

129/129 [==============================] - 1s 11ms/step


,Threshold,positive_count,F1,Precision,Recall,AUC,Accuracy
0,0.00,[4108],0.072719,0.037731,1.000000,0.500000,0.037731
1,0.05,[674],0.221954,0.136499,0.593548,0.723159,0.842989
2,0.10,[568],0.224066,0.142606,0.522581,0.699692,0.863437
3,0.15,[506],0.223903,0.146245,0.477419,0.684068,0.875122
4,0.20,[451],0.231023,0.155211,0.451613,0.677615,0.886563
5,0.25,[402],0.222621,0.154229,0.400000,0.656995,0.894596
6,0.30,[370],0.220952,0.156757,0.374194,0.647633,0.900438
7,0.35,[342],0.229376,0.166667,0.367742,0.647822,0.906767
8,0.40,[314],0.217484,0.162420,0.329032,0.631250,0.910662
9,0.45,[290],0.211236,0.162069,0.303226,0.620877,0.914557


## Map Raw to Patient

In [ ]:
def output_mapper(model, dataset, idx, x, chosen_threshold, y_true):
  
  keys = list(idx.keys())

  # Indexing referenced relative to the FULL dataset (i.e., index 0 to 27k)
  if dataset == 'train':
    start_key_pos = 0
    end_key_pos = np.int(0.7*len(keys))
    sliced_keys = keys[start_key_pos : end_key_pos+1]
    
  elif dataset == 'val':
    start_key_pos = np.int(0.7*len(keys))+1
    end_key_pos = np.int(0.85*len(keys))
    sliced_keys = keys[start_key_pos : end_key_pos+1]
    # print(sliced_keys)

  elif dataset == 'test':
    start_key_pos = np.int(0.85*len(keys))+1
    sliced_keys = keys[start_key_pos : len(keys)]

  y_pred_probs = model.predict(x)
  y_pred = np.where(y_pred_probs >= chosen_threshold, 1, 0)
  true_y_mapped = []
  pred_y_mapped = []

  for i in range(0, len(sliced_keys)):

    # Extract indexing for start and end image per patient
    start_img_idx = idx[sliced_keys[i]][0]    # 0
    end_img_idx = idx[sliced_keys[i]][1]      # 31
    # print(start_img_idx, end_img_idx)

    # Slice y true and predicted
    sliced_y = y_true[start_img_idx : end_img_idx]

    if dataset == 'val' or dataset == 'test':
      start_img_idx = start_img_idx - idx[sliced_keys[0]][0]
      end_img_idx = end_img_idx - idx[sliced_keys[0]][0]
      # print(start_img_idx, end_img_idx)
    
    sliced_y_pred = y_pred[start_img_idx : end_img_idx]

    # Store in lists
    if 1 in sliced_y:
      true_y_mapped.append(1)
    else:
      true_y_mapped.append(0)
    
    if 1 in sliced_y_pred:
      pred_y_mapped.append(1)
    else:
      pred_y_mapped.append(0)  

  return true_y_mapped, pred_y_mapped  

In [ ]:
true_y_train_mapped, pred_y_train_mapped = output_mapper(model_final, 'train', idx, x_train, 0.05, y)
print(len(true_y_train_mapped), len(pred_y_train_mapped))

599/599 [==============================] - 5s 9ms/step
864 864


In [ ]:
true_y_val_mapped, pred_y_val_mapped = output_mapper(model_final, 'val', idx, x_val, 0.05, y)
print(len(true_y_val_mapped), len(pred_y_val_mapped))

128/128 [==============================] - 1s 9ms/step
185 185


In [ ]:
true_y_test_mapped, pred_y_test_mapped = output_mapper(model_final, 'test', idx, x_test, 0.05, y)
print(len(true_y_test_mapped), len(pred_y_test_mapped))

129/129 [==============================] - 1s 9ms/step
184 184


In [ ]:
accuracy_train = accuracy_score(true_y_train_mapped, pred_y_train_mapped)
precision_train = precision_score(true_y_train_mapped, pred_y_train_mapped)
recall_train = recall_score(true_y_train_mapped, pred_y_train_mapped)
f1_train = f1_score(true_y_train_mapped, pred_y_train_mapped)
auc_train = roc_auc_score(true_y_train_mapped, pred_y_train_mapped)

print("----- TRAIN -----")
print(f"F1 Score: {f1_train:.4f}")
print(f"Precision: {precision_train:.4f}")
print(f"Recall: {recall_train:.4f}")
print(f"AUC: {auc_train:.4f}")
print(f"Accuracy: {accuracy_train:.4f}")
print("\n")

accuracy_val = accuracy_score(true_y_val_mapped, pred_y_val_mapped)
precision_val = precision_score(true_y_val_mapped, pred_y_val_mapped)
recall_val = recall_score(true_y_val_mapped, pred_y_val_mapped)
f1_val = f1_score(true_y_val_mapped, pred_y_val_mapped)
auc_val = roc_auc_score(true_y_val_mapped, pred_y_val_mapped)

print("----- VALIDATION -----")
print(f"F1 Score: {f1_val:.4f}")
print(f"Precision: {precision_val:.4f}")
print(f"Recall: {recall_val:.4f}")
print(f"AUC: {auc_val:.4f}")
print(f"Accuracy: {accuracy_val:.4f}")
print("\n")

accuracy_test = accuracy_score(true_y_test_mapped, pred_y_test_mapped)
precision_test = precision_score(true_y_test_mapped, pred_y_test_mapped)
recall_test = recall_score(true_y_test_mapped, pred_y_test_mapped)
f1_test = f1_score(true_y_test_mapped, pred_y_test_mapped)
auc_test = roc_auc_score(true_y_test_mapped, pred_y_test_mapped)

print("----- TEST -----")
print(f"F1 Score: {f1_test:.4f}")
print(f"Precision: {precision_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"AUC: {auc_test:.4f}")
print(f"Accuracy: {accuracy_test:.4f}")
print("\n")

# Define the metrics and corresponding values
metrics = ["F1 Score", "Precision", "Recall", "AUC", "Accuracy"]
train_scores = [f1_train, precision_train, recall_train, auc_train, accuracy_train]
val_scores = [f1_val, precision_val, recall_val, auc_val, accuracy_val]
test_scores = [f1_test, precision_test, recall_test, auc_test, accuracy_test]

# Create the dataframe
data = {"Train": train_scores, "Validation": val_scores, "Test": test_scores}
df = pd.DataFrame(data, index=metrics)

# Print the dataframe
df.transpose()

----- TRAIN -----
F1 Score: 0.3439
Precision: 0.2115
Recall: 0.9196
AUC: 0.7045
Accuracy: 0.5451


----- VALIDATION -----
F1 Score: 0.4459
Precision: 0.2895
Recall: 0.9706
AUC: 0.7171
Accuracy: 0.5568


----- TEST -----
F1 Score: 0.4355
Precision: 0.2872
Recall: 0.9000
AUC: 0.7325
Accuracy: 0.6196




,F1 Score,Precision,Recall,AUC,Accuracy
Train,0.343907,0.211499,0.919643,0.704502,0.545139
Validation,0.445946,0.289474,0.970588,0.717082,0.556757
Test,0.435484,0.287234,0.900000,0.732468,0.619565


## Debugging

In [ ]:
start_key_pos = np.int(0.85*len(keys))+1
end_key_pos = len(keys)
print('Starting key position =', start_key_pos, '| End key position =', end_key_pos)
sliced_keys = keys[start_key_pos : end_key_pos+1]
print('Patient IDs =', sliced_keys)
i = -1
start_img_idx = idx[sliced_keys[i]][0] #- idx[sliced_keys[0]][0]   # 0
end_img_idx = idx[sliced_keys[i]][1] #- idx[sliced_keys[0]][0]    # 31
print('Start image index =', start_img_idx, '| End image index =', end_img_idx)

Starting key position = 1049 | End key position = 1233
Patient IDs = ['11257_1001280', '11259_1001282', '11260_1001283', '11261_1001284', '11262_1001285', '11263_1001286', '11264_1001287', '11265_1001288', '11267_1001290', '11268_1001291', '11269_1001292', '11270_1001293', '11271_1001294', '11272_1001295', '11273_1001296', '11275_1001298', '11276_1001299', '11278_1001301', '11279_1001302', '11280_1001303', '11281_1001304', '11282_1001305', '11284_1001307', '11286_1001309', '11287_1001310', '11288_1001311', '11289_1001312', '11290_1001313', '11291_1001314', '11292_1001315', '11293_1001316', '11294_1001317', '11295_1001318', '11297_1001320', '11298_1001321', '11299_1001322', '11301_1001324', '11303_1001326', '11304_1001327', '11305_1001328', '11306_1001329', '11307_1001330', '11308_1001331', '11309_1001332', '11310_1001333', '11311_1001334', '11312_1001335', '11314_1001337', '11315_1001338', '11316_1001339', '11317_1001340', '11318_1001341', '11319_1001342', '11320_1001343', '11321_10013

In [ ]:
x = 27342  # Value to search for

found_key = None

for key, value in idx.items():
    if value[1] == x:
        found_key = key
        break

print(found_key)

11474_1001498
